In [44]:
import urllib.request
import json
import numpy as np
import pandas as pd
from joblib import load
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [45]:
def custom_analyzer(doc):
    return doc

cardVec = load('matrix_data/cardVec_4k_20dim.joblib')
deckVecorizer = load('matrix_data/deckVectorizer_4k_15K.joblib')

In [46]:
data_url = 'https://raw.githubusercontent.com/KU81-ComSci-ProjectEnd-GameCard/main/master/Predictor/data/processed_data/picked_4k_15k.json'
with urllib.request.urlopen(data_url) as url:
    pick_js = json.loads(url.read().decode())
pick = pd.DataFrame(pick_js)

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

In [48]:
X = deckVecorizer.fit_transform(pick.deck.tolist())
X = X.toarray()
y = [cardVec[picked] for picked in pick.picked ]
y_card = pick.picked 

# Convert your data into numpy arrays and standardize
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)
y_card = np.array(y_card, dtype=str)

In [49]:
X_train, X_test, y_train, y_test, yC_train, yC_test = train_test_split(X, y, y_card, test_size=0.2, random_state=42)

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [35]:
# Create DataLoader for training and testing
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch = 500
train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True,num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=False,num_workers=4)

In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(257, 128)  # Input layer
        self.fc2 = nn.Linear(128, 64)   # First hidden layer
        self.fc3 = nn.Linear(64, 20)    # Second hidden layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Should print True if CUDA is available
print(torch.version.cuda)

2.4.0+cu118
True
11.8


In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNetwork()
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [42]:
scaler = torch.amp.GradScaler()
def train_model(model, criterion, optimizer, train_loader, epochs=600):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            with torch.amp.autocast(device_type='cuda'): # Enable mixed precision
                outputs = model(inputs)
                loss = criterion(outputs, targets)
            scaler.scale(loss).backward()  # Scale the loss
            scaler.step(optimizer)  # Update weights
            scaler.update()

            running_loss += loss.item()
        # Print loss every 100 epochs
        if (epoch + 1) % 1 == 0:
            print(f'Epoch {epoch + 1}, Loss: {running_loss/len(train_loader):.4f}')


In [43]:
train_model(model, criterion, optimizer, train_loader)

Epoch 1, Loss: 0.3737
Epoch 2, Loss: 0.3111
Epoch 3, Loss: 0.3019
Epoch 4, Loss: 0.2979
Epoch 5, Loss: 0.2951
Epoch 6, Loss: 0.2925
Epoch 7, Loss: 0.2905
Epoch 8, Loss: 0.2884
Epoch 9, Loss: 0.2865
Epoch 10, Loss: 0.2841
Epoch 11, Loss: 0.2820
Epoch 12, Loss: 0.2804
Epoch 13, Loss: 0.2776
Epoch 14, Loss: 0.2758
Epoch 15, Loss: 0.2736
Epoch 16, Loss: 0.2712
Epoch 17, Loss: 0.2689
Epoch 18, Loss: 0.2666
Epoch 19, Loss: 0.2645
Epoch 20, Loss: 0.2619
Epoch 21, Loss: 0.2597
Epoch 22, Loss: 0.2573
Epoch 23, Loss: 0.2551
Epoch 24, Loss: 0.2528
Epoch 25, Loss: 0.2508


KeyboardInterrupt: 